In [7]:
# Initialization

import math
from datetime import datetime
import openpyxl as opxl
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import model as ty
import proc as pr
import matplotlib.ticker as mtick
import sklearn.metrics

import sys
sys.path.append('../RD-systems-and-test-benches')

import utils.plot_functions as RD_pfun
import config as RD_config

from IPython.core.display import HTML

import heat_transfer as bht

import fluids as fds
import ht 

import general as gen

import os

import scipy.integrate as integrate
import scipy.optimize as sco

import networkx as nx

import plotly.graph_objects as go

import plot_functions_here as pfun

import plotly.io as pio

from itertools import product

from tqdm.notebook import tqdm

In [8]:
folder_path = r'G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-perf-1Dmodel'
geometry_path = folder_path + '\\' + 'Panels\\Panel_V4.1I.xlsx'
hypotheses_path = folder_path + '\\' + 'Hypotheses\\Model_hypotheses.xlsx'
condi_path = folder_path + '\\' + 'Steady-state conditions inputs\\100424-tests-conditions-GMI.xlsx'
exp_path = folder_path + '\\' + 'Experiments\\Experiments-GMI.xlsx'

panelSpecs = pr.import_geometry(geometry_path)
hyp = pr.create_dict_from_excel(hypotheses_path,'Main')

# steadyStateConditions_df = pd.read_excel(condi_path,header=2) ## Les deux première lignes ne sont pas à prendre en compte
# stepConditions = steadyStateConditions_df.to_dict(orient='records')

In [9]:
recap = pd.read_excel(r'G:\Drive partagés\Cercle Hard\R&D\Thermal performance tests PVT\Recap_RD_TUV_SSI.xlsx')
recap = recap.drop(index=0)
recap.dropna(how='all', inplace=True)

groups = {key: group for key, group in recap.groupby('Design')}

In [10]:
steadyStateConditions_df = groups['V4.1I'].loc[groups['V4.1I']['Type'] == 'SK'].copy()
pr.convert_to_K(steadyStateConditions_df)
pr.adjust_steadyStateConditions(steadyStateConditions_df, hyp)
steadyStateConditions_df.reset_index(drop=True, inplace=True)
steadyStateConditions = steadyStateConditions_df.to_dict(orient='records')

In [11]:
# df_one,res = ty.simu_one_steady_state_all_he(panelSpecs,sC,hyp)
# df_residuals = ty.recap_residuals(panelSpecs, df_one, res)

In [12]:
df_res, list_res = ty.simu_steadyStateConditions(panelSpecs,hyp,steadyStateConditions_df)

In [13]:
df_res['Qdot_exp'] = steadyStateConditions_df['Qdot']
df_res['error'] = df_res['Qdot_tube_fluid'] - df_res['Qdot_exp']

In [23]:
df = df_res.copy()

df = df.loc[df['u'] == 2.7]

ht_signs = {'Qdot_sun_PV': 1, 'Qdot_sun_glass': 1, 'Qdot_top_conv': -1, 'Qdot_top_rad': -1, 'Qdot_PV_sky': -1, 'Qdot_absfin_back_conv': -1, 'Qdot_absfin_back_rad': -1, 'Qdot_Base_back': -1, 'Qdot_tube_back_conv': -1, 'Qdot_tube_back_rad': -1, 'Qdot_tube_sky': -1}
ht_labels = ['Qdot_sun_PV', 'Qdot_sun_glass', 'Qdot_top_conv', 'Qdot_top_rad', 'Qdot_PV_sky', 'Qdot_absfin_back_conv', 'Qdot_absfin_back_rad', 'Qdot_tube_back_conv', 'Qdot_tube_back_rad', 'Qdot_tube_sky']
colors_ht = [RD_config.get_hex_code('yellow',100), RD_config.get_hex_code('orange',100),RD_config.get_hex_code('green',150), RD_config.get_hex_code('blue',150),RD_config.get_hex_code('blue',100),RD_config.get_hex_code('green',100),RD_config.get_hex_code('grey',100),RD_config.get_hex_code('green',50),RD_config.get_hex_code('grey',50),RD_config.get_hex_code('blue',50)]

# Calculate the total positive heat transfers for each T_m - T_amb
df['positive_total'] = df[[label for label in ht_labels if ht_signs[label] == 1]].sum(axis=1)

# Initialize the figure
fig = go.Figure()

# Define the bar width
bar_width = 0.4

# Add traces for positive heat transfers (gains)
for i, label in enumerate(ht_labels):
    if ht_signs[label] == 1:
        fig.add_trace(go.Bar(
            x=df['T_m - T_amb'],
            y=df[label],
            name=label,
            width=[bar_width] * len(df),
            hovertext=round(df[label],0),
            hoverinfo='text',
            marker=dict(color=colors_ht[i])
        ))

# Add traces for negative heat transfers (losses) with base starting from the positive total
label_sum = []
for i, label in enumerate(ht_labels):
    if ht_signs[label] == -1:
        base = df['positive_total'] - df[[lbl for lbl in ht_labels if (ht_signs[lbl] == -1) & (lbl not in label_sum)]].sum(axis=1)
        fig.add_trace(go.Bar(
            x=df['T_m - T_amb'] + bar_width,  # Offset for side-by-side display
            y=df[label],
            name=label,
            width=[bar_width] * len(df),
            base=base,
            hovertext=round(df[label],0),
            hoverinfo='text',
            marker=dict(color=colors_ht[i])
        ))
        label_sum.append(label)


fig.add_trace(go.Scatter(
    x=df['T_m - T_amb'],
    y=df['Qdot_exp'],
    mode='lines',
    name='Qdot exp',
    line=dict(color='black', width=2)
))

fig.add_trace(go.Scatter(
    x=df['T_m - T_amb'],
    y=df['Qdot_tube_fluid'],
    mode='lines',
    name='Qdot exp',
    line=dict(color='grey', width=2)
))

# Update layout for stacked bars
fig.update_layout(barmode='relative', yaxis_title='Heat Transfer', xaxis_title='T_m - T_amb')


# Show the figure
fig.show()

In [ ]:
df['T_m - T_amb']

In [ ]:
df['negative_total']

In [ ]:
# df_res[['Qdot_tube_fluid','Qdot_exp','error']]

In [ ]:
mae = np.abs(df_res['error']).mean()
print(mae)

In [ ]:
wm = 7

# Deuxième écriture plus simple pour modifier les coefficients
mapping_matrix = np.array([
                            wm*[''],
                            ['tau_g', 'alpha_g', 'alpha_PV', 'eps_PV', 'eps_g', 'l_c', 'lambd_air'],
                           ])

part_mapping_matrix = [
                        np.nan,
                        [['part1','part2','part3'] for _ in range(wm)]
                        ]

ranges_matrix = np.array(list(product(np.arange(0.80,0.98,0.02),
                                        np.array([0.05]),
                                        np.array([0.95]),
                                        np.array([0.9]),
                                        np.array([0.9]),
                                        np.array([0.5e-3,1e-3,1.5e-3]),
                                        np.array([0.5e-3,1e-3,1.5e-3])
                                        )))

In [24]:
wm = 1

# Deuxième écriture plus simple pour modifier les coefficients
mapping_matrix = np.array([
                            wm*[''],
                            ['lambd_air'],
                           ])

part_mapping_matrix = [
                        np.nan,
                        [['part1','part2','part3'] for _ in range(wm)]
                        ]

ranges_matrix = np.array(list(product(
                                        np.array([0.1e-3,10e-3,1.1e-4])
                                        )))

In [ ]:
ranges_matrix

In [ ]:
part_mapping_matrix[1]

In [ ]:
df_res_list = []
mae_list = []
rmse_list = []

index_geometry = 1

for i in tqdm(range(ranges_matrix.shape[0])):
    for j in range(ranges_matrix.shape[1]):
        for k, part_name in enumerate(part_mapping_matrix[index_geometry][j]):
            panelSpecs[part_name][mapping_matrix[index_geometry,j]] = ranges_matrix[i,j]
    df_res, list_res = ty.simu_steadyStateConditions(panelSpecs,hyp,steadyStateConditions_df)

    df_res['Qdot exp'] = steadyStateConditions_df['Qdot']
    df_res['error'] = df_res['Qdot_tube_fluid'] - df_res['Qdot exp']
    df_res['relative error'] = (df_res['Qdot_tube_fluid'] - df_res['Qdot exp'])/df_res['Qdot exp']
    df_res['squared error'] = (df_res['Qdot_tube_fluid'] - df_res['Qdot exp'])**2
    mae = df_res['error'].abs().mean()
    rmse = np.sqrt(df_res['squared error'].mean())

    df_res_list.append(df_res)
    mae_list.append(mae)
    rmse_list.append(rmse)

In [ ]:
df_res_list[9][['T_amb','u','T_fluid_in','Qdot_tube_fluid','Qdot exp','error']]

In [ ]:
min_index = mae_list.index(min(mae_list))
print(min_index)

In [ ]:
min(mae_list)

In [ ]:
df_res_list[3][['Qdot_tube_fluid','Qdot exp','error']]

In [ ]:
mae_list

In [ ]:
plt.scatter(np.arange(0,len(rmse_list),1),rmse_list)

In [ ]:
panelSpecs['pv']['tau_g']

In [ ]:
len(ranges_matrix)*2/3600

In [ ]:
all_combinations = list(product(range1, range2, range3))
matrix = np.array(all_combinations)

In [ ]:
matrix

Calibrage modèle multi-design

In [ ]:
# Permet de modifier une valeur du dictionnaire hyp
## Par exemple init_hyp('method_h_back_abs','free') va retourner le dictionnaire hyp avec la valeur 'method_h_back_abs' modifié en 'free'
def init_hyp(coeff, method) : 
    hyp[coeff] = method
    return(hyp)

Création de la matrice du modèle

In [ ]:
# Récupération des résultats expérimentaux

X_list_df = pd.read_excel(exp_path,header = 2) ## On ne prend pas en compe les 2 premières lignes // EN FONCTION DE COMMENT SONT PR2SENT2S LES RES
X_list = X_list_df.to_dict(orient='records')

In [ ]:
# Création de la matrice "A" sous forme de dictionnaire

measured_case = 0 # nb_measured_case = len(X_tot)
case = 0 # nb_case = len(stepConditions) il faudrait faire correspondre les lignes de stepConditions avec le numéro des measuredcase comme ça on a qu'une variable

# Dictionnaire des paramètres du modèle commun à tous les designs "alpha_0"
basic_parameters = dict([
    ('basic_parameters_1',1),
    ('basic_parameters_2',1),
    ('basic_parameters_3',1),
    ('basic_parameters_4',1),
    ('basic_parameters_5',1),
    ('basic_parameters_6',1),
    ('basic_parameters_7',1),
    ('basic_parameters_8',1),
    ('basic_parameters_9',1),
    ('basic_parameters_10',1),
    ])

# Matrice du modèle "A" 
model_matrix = dict(['alpha_0',basic_parameters])

# Plutôt faire l'inverse et créer le dictionnaire à partir d'une DataFrame puis sortir les différents X
# for i in range(nb_measured_case) :
# Point de données
X = dict([('componentSpecs',componentSpecs), ('stepConditions',stepConditions[case])]) 
# X_tot = dict([ 
#    ('X_{measured_case}',X)
#   ])

# Dictionnaire des paramètres du modèle liés à la géométrie "alpha_j"
design_parameters = dict([
    ('tau_g_{measured_case}', X['componentSpecs']['pv']['tau_g']), # Transmittance glass
    ('alpha_g_{measured_case}', X['componentSpecs']['pv']['alpha_g']), # Absorptance glass
    ('eps_PV_{measured_case}', X['componentSpecs']['pv']['eps_PV']), # Émissivité cellule PV
    ('eps_g_{measured_case}', X['componentSpecs']['pv']['eps_g']), # Émissivité glass
    ('eps_hx_top_{measured_case}', X['componentSpecs']['main']['eps_hx_top']), # Émissivité de l'échangeur thermique sur le panneau
    ('eps_hx_back_{measured_case}', X['componentSpecs']['main']['eps_hx_back']), # Émissivité de l'échangeur thermique à l’arrière du panneau
    ('eps_ins_{measured_case}', X['componentSpecs']['main']['eps_ins']), # Émissivité isolant
    ('lambd_air_{measured_case}',  X['componentSpecs']['main']['lambd_air']) # Épaisseur de la couche d'air
    ])

model_matrix['alpha_{measured_case}'] = design_parameters
#fin boucle for

In [ ]:
# Création de la matrice "A" sous forme de listes

measured_case = 0 # nb_measured_case = len(X_list)
case = 0 # nb_case = len(stepConditions) il faudrait faire correspondre les lignes de stepConditions avec le numéro des measuredcase comme ça on a qu'une variable

# Matrice du modèle "A" 
model_matrix = np.ones(10)

# Plutôt faire l'inverse et créer le dictionnaire à partir d'une DataFrame puis sortir les différents X
# for i in range(nb_measured_case) :
# Point de données
X = dict([('componentSpecs',componentSpecs), ('stepConditions',stepConditions[case])]) 
# X_list = dict([ 
#    ('X_{measured_case}',X)
#   ])

# Dictionnaire des paramètres du modèle liés à la géométrie "alpha_j"
design_parameters = np.zeros(10)
design_parameters[0] = X['componentSpecs']['pv']['tau_g'] # Transmittance glass
design_parameters[1] =  X['componentSpecs']['pv']['alpha_g'] # Absorptance glass
design_parameters[2] = X['componentSpecs']['pv']['eps_PV'] # Émissivité cellule PV
design_parameters[3] = X['componentSpecs']['pv']['eps_g'] # Émissivité glass
design_parameters[4] = X['componentSpecs']['main']['eps_hx_top'] # Émissivité de l'échangeur thermique sur le panneau
design_parameters[5] = X['componentSpecs']['main']['eps_hx_back'] # Émissivité de l'échangeur thermique à l’arrière du panneau
design_parameters[6] = X['componentSpecs']['main']['eps_ins'] # Émissivité isolant
design_parameters[7] = X['componentSpecs']['main']['lambd_air'] # Épaisseur de la couche d'air


model_matrix= np.vstack((model_matrix, design_parameters))

#fin boucle for
model_matrix

In [ ]:
m = len(correspondance_matrice)
nb_measured_case = len(X_list)

# MAJ de hyp en fonction des méthodes que l'on veut
hyp = init_hyp("method_h_back_abs", "free")


def Qdot_model_tube_fluid(X_list, model_matrix, correspondance_matrice) :
        Qdot_list = []
        
        # Actualise le modèle
        for i in range(m) : 
                if correspondance_matrice[i, 0] != 0. : 
                       hyp[correspondance_matrice[i,0]] = model_matrix[i, 0]

        # Actualise la géométrie
        for j in range(nb_measured_case) : 
                for i in range(m) :
                        if correspondance_matrice[i, 0] != 0. :
                                X_list[j]['componentSpecs'][correspondance_matrice[i,1]] = model_matrix[i, j]
                df_one = ty.simu_one_steady_state_all_he(X_list[j]['componentSpecs'], X_list[j]['stepConditions'], hyp)[0]
                Qdot_list.append(df_one['Qdot_tube_fluid'])
        return(Qdot_list)